In [10]:
!pip install osmnx rasterio

In [11]:
!pip install earthengine-api geemap


In [12]:
!pip install geopy


In [4]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import osmnx as ox
from geopy.geocoders import Nominatim
from shapely.geometry import LineString
import cv2
import os
import time
from geopy.geocoders import Nominatim
import ee
from PIL import Image
import pandas as pd

In [3]:
!pip install earthengine-api geemap rasterio osmnx geopy shapely opencv-python-headless pandas matplotlib numpy


In [5]:

# list of cities
# Read the CSV file
df = pd.read_csv("C:/Users/DELL/OneDrive/Belgeler/GitHub/Applied_research_studio/rcs/cities_list.csv")
print(df)
# Assume the column you want to convert to a list is named 'your_column'
cities_list = df['City, Country'].tolist()



          City, Country
0          Delhi, India
1       Shanghai, China
2     Sao Paulo, Brazil
3          Cairo, Egypt
4         Mumbai, India
..                  ...
990  Gothenburg, Sweden
991      Seville, Spain
992     Zagreb, Croatia
993        Lyon, France
994        Nice, France

[995 rows x 1 columns]


In [6]:

# Define save directory
SAVE_DIR = "C:/Users/DELL/OneDrive/Masaüstü/DATASET ALIGNED"
os.makedirs(SAVE_DIR, exist_ok=True)

In [7]:
#_____________________________________________________________________________________________________________________________________________________________________

## gee credentials

# Authenticate and initialize
ee.Authenticate()  # Run this only once; follow the on-screen instructions
# Explicitly set the project ID
ee.Initialize(project='ee-misraserenayozgok') # Replace 'your-project-id' with your actual Google Cloud project ID


In [8]:
# Function to get OSM boundary
def get_osm_boundary(city_name):
    geolocator = Nominatim(user_agent="geoapi", timeout=10)
    try:
        location = geolocator.geocode(city_name, exactly_one=True)
        if location:
            bbox = location.raw.get('boundingbox', None)
            if bbox:
                lat_min, lat_max, lon_min, lon_max = map(float, bbox)
                return {
                    "bbox": (lon_min, lat_min, lon_max, lat_max),
                    "gee_rectangle": ee.Geometry.BBox(lon_min, lat_min, lon_max, lat_max)
                }
    except Exception as e:
        print(f"Error fetching bbox for {city_name}: {e}")
    return None

# LOAD OSMNX

In [9]:
# Function to generate overlapped map
def generate_aligned_map(city_name):
    print(f"\n🌍 Processing: {city_name}...")
    boundary_data = get_osm_boundary(city_name)
    if not boundary_data:
        print(f"❌ Could not retrieve boundary for {city_name}, skipping...")
        return

    # Get OSM street data
    G = ox.graph_from_bbox(boundary_data["bbox"], network_type='drive')
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

    # Get NASADEM elevation data
    layer = "NASA/NASADEM_HGT/001"
    band = "elevation"
    image = ee.Image(layer).select(band)
    array = geemap.ee_to_numpy(image, region=boundary_data["gee_rectangle"], bands=[band])

    if array is None:
        print(f"❌ No elevation data available for {city_name}, skipping...")
        return

    # Normalize elevation data
    array = array.astype(np.float32) / np.max(array)  # Normalize

    # Plot the overlay
    fig, ax = plt.subplots(figsize=(10, 10))
    minx, miny, maxx, maxy = boundary_data["bbox"]
    ax.imshow(array, extent=[minx, maxx, miny, maxy], origin="upper", cmap="gray")
    gdf_edges.plot(ax=ax, color="red", linewidth=0.5, alpha=1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # Save the map
    save_path = os.path.join(SAVE_DIR, f"{city_name.replace(',', '').replace(' ', '_')}_aligned.png")
    plt.savefig(save_path, dpi=400, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"✅ Saved: {save_path}")

# Load GEE data

# PLOT

In [ ]:
# Process each city in the list
for city in cities_list:
    generate_aligned_map(city)
    time.sleep(2)  # Avoid API rate limits

print("\n🎉 Dataset generation complete!")


🌍 Processing: Delhi, India...
